In [3]:
import pandas as pd

df = pd.read_csv('Learners_Digital_raw.csv')
df.head()

,User_ID,Nom,Cours,Temps_Connexion_h,Activités_Complétées,Score_Quiz_%,Feedback,Certifié
0,U014,Yacine,IA Santé,10,4.0,47.1,Excellent,yes
1,U016,Reda,Web Dev,6,2.0,72.2,bof,yes
2,U023,Yasmine,Python DS,10,NaN,43%,Moyen,Non
3,U037,Mounia,IA Santé,13,3.0,73.3,Moyen,Non
4,U041,Walid,Web Dev,7,8.0,62.9,Moyen,N


In [9]:
df.tail()

,User_ID,Nom,Cours,Temps_Connexion_h,Activités_Complétées,Score_Quiz_%,Feedback,Certifié
42,U024,Amina,IA Santé,11,8.0,62.3,Faible,Oui
43,U020,Nour,Data Viz,4,4.0,79.9,Moyen,Non
44,U004,Nour,Python DS,11,7.0,58%,Faible,no
45,U026,Asmaa,IA Santé,8,4.0,70.1,Bon,Non
46,U005,Nesrine,Python DS,8,4.0,69.3,Excellent,Non


User_ID:identifiant de l'apprenant
Nom: Le nom de l'apprenant 
cours:l'abreviation du cours 
temps_Connexion_h:le temps consacrer pour la revision
Activité_Complétées:


#Probleme a regle lors du nettoyage
valeurs manquantes probleme de pourcentage no,non .....


le dataset contient 47 apprennent (ligne)qui sont caractetriser par 8 variables 

In [10]:
print('Dimension :' ,df.shape)


Dimension : (47, 8)


des valeurs manquantes , probeleme de types

In [11]:
print('\n les infos :')
print(df.info())



 les infos :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   User_ID               47 non-null     object 
 1   Nom                   47 non-null     object 
 2   Cours                 47 non-null     object 
 3   Temps_Connexion_h     47 non-null     object 
 4   Activités_Complétées  43 non-null     float64
 5   Score_Quiz_%          44 non-null     object 
 6   Feedback              43 non-null     object 
 7   Certifié              45 non-null     object 
dtypes: float64(1), object(7)
memory usage: 3.1+ KB
None


In [12]:
print('Describe ')
print(df.describe())


Describe 
       Activités_Complétées
count             43.000000
mean               5.488372
std                2.354045
min                1.000000
25%                4.000000
50%                6.000000
75%                7.500000
max                9.000000


 Nettoyage du dataset 
 

In [13]:
before=len(df)
df['Nom']=df['Nom'].astype(str).str.strip()
df.drop_duplicates(inplace=True)
print('les doublons supprimés :' , before - len(df))


les doublons supprimés : 2


In [14]:
df['Temps_Connexion_h'] = pd.to_numeric(df['Temps_Connexion_h'].astype(str).str.replace('h','',regex=False), errors='coerce')
df['Activités_Complétées'] = pd.to_numeric(df['Activités_Complétées'], errors='coerce')
df['Score_Quiz_%'] = pd.to_numeric(df['Score_Quiz_%'].astype(str).str.replace('%','',regex=False), errors='coerce')


In [15]:

map_feed = {'EXELLENT':'Excellent','EXCELLENT':'Excellent','BON':'Bon','MOYEN':'Moyen','FAIBLE':'Faible','BOF':'Faible','':pd.NA}
df['Feedback'] = df['Feedback'].astype(str).str.strip().str.upper().map(map_feed)



In [16]:
map_cert = {'OUI':'Oui','O':'Oui','YES':'Oui','NON':'Non','N':'Non','NO':'Non'}
df['Certifié'] = df['Certifié'].astype(str).str.strip().str.upper().map(map_cert)



In [17]:
df['Réussite'] = df.apply(lambda r: 'Oui' if (r['Score_Quiz_%']>=70) and (r['Activités_Complétées']>=5) else 'Non', axis=1)
df.head()

,User_ID,Nom,Cours,Temps_Connexion_h,Activités_Complétées,Score_Quiz_%,Feedback,Certifié,Réussite
0,U014,Yacine,IA Santé,10,4.0,47.1,Excellent,Oui,Non
1,U016,Reda,Web Dev,6,2.0,72.2,Faible,Oui,Non
2,U023,Yasmine,Python DS,10,NaN,43.0,Moyen,Non,Non
3,U037,Mounia,IA Santé,13,3.0,73.3,Moyen,Non,Non
4,U041,Walid,Web Dev,7,8.0,62.9,Moyen,Non,Non


Analyse grouper 

In [18]:
print('Moyenne du Score par Cours:')
print(df.groupby('Cours')['Score_Quiz_%'].mean().round(1))

print('\nTaux de Réussite par Cours (%):')
print((df.groupby('Cours')['Réussite'].apply(lambda s: (s=='Oui').mean()*100)).round(1))

print('\nTaux de Certification par Feedback (%):')
print((df.groupby('Feedback')['Certifié'].apply(lambda s: (s=='Oui').mean()*100)).round(1))

Moyenne du Score par Cours:
Cours
Data Viz     81.0
IA Santé     63.3
Python DS    62.7
Web Dev      63.4
Name: Score_Quiz_%, dtype: float64

Taux de Réussite par Cours (%):
Cours
Data Viz     25.0
IA Santé      6.7
Python DS    17.6
Web Dev      22.2
Name: Réussite, dtype: float64

Taux de Certification par Feedback (%):
Feedback
Bon          20.0
Excellent    40.0
Faible       37.5
Moyen        30.8
Name: Certifié, dtype: float64


In [19]:
Corrélation Temps ↔ Score (cercle -1..+1)

In [20]:
sub = df[['Temps_Connexion_h','Score_Quiz_%']].dropna()
r = sub['Temps_Connexion_h'].corr(sub['Score_Quiz_%'])
print(f"Corrélation (Temps ↔ Score) : r = {r:.3f}")

Corrélation (Temps ↔ Score) : r = 0.030


In [6]:
df.to_csv('Learners_Digital_clean.csv', index=False)
print('Exporté : Learners_Digital_clean.csv')

Exporté : Learners_Digital_clean.csv


1. Quel est le cours le plus performant ?
2. Quel feedback est le plus associé à la certification ?
3. Quelle est la corrélation Activités_Complétées ↔ Score (et son interprétation sur le cercle) ?

data viz
exellent
r=0.030